In [3]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as image
import os
import shutil
import random
import re
import tensorflow as tf
import numpy as np
import json
import time
import sklearn.model_selection

In [4]:
classes = ['shoe','trousers','jacket','sweatshirt']

In [5]:
def getModel(num_classes):
    basemodel = efb3 = tf.keras.applications.EfficientNetB3(
        weights="imagenet",
        include_top=False
    )
    basemodel.trainable = False
    model = tf.keras.models.Sequential()
    model.add(basemodel)
    model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    return model

In [6]:
mymodel = getModel(len(classes))
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, None, None, 1536)  10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 6148      
Total params: 10,789,683
Trainable params: 6,148
Non-trainable params: 10,783,535
_________________________________________________________________


In [33]:
mymodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
jackets_filtered = pd.read_csv('jackets_filtered.csv', index_col=0)
jackets_filtered["class"] = "jacket"
jackets_filtered.head()

,fname,class
0,1085.jpg,jacket
1,1086.jpg,jacket
2,1087.jpg,jacket
3,1382.jpg,jacket
4,1383.jpg,jacket


In [9]:
scarpe_filtered = pd.read_csv('scarpe_filtered.csv', index_col=0)
scarpe_filtered["class"] = "shoe"
scarpe_filtered.head()

,fname,class
0,15319.jpg,shoe
1,15324.jpg,shoe
2,15347.jpg,shoe
3,15356.jpg,shoe
4,15370.jpg,shoe


In [10]:
sweatshirt_filtered = pd.DataFrame(os.listdir('../sweatshirt'), columns=["fname"])
sweatshirt_filtered["class"] = "sweatshirt"
sweatshirt_filtered.head()

,fname,class
0,5607.jpg,sweatshirt
1,6133.jpg,sweatshirt
2,6542.jpg,sweatshirt
3,6127.jpg,sweatshirt
4,375.jpg,sweatshirt


In [11]:
pantaloni_filtered = pd.DataFrame(os.listdir('../pantaloni'), columns=["fname"])
pantaloni_filtered["class"] = "trousers"
pantaloni_filtered.head()

,fname,class
0,7911.jpg,trousers
1,6888.jpg,trousers
2,7345.jpg,trousers
3,7448.jpg,trousers
4,9012.jpg,trousers


In [12]:
filtered_df = pd.concat([scarpe_filtered, pantaloni_filtered, sweatshirt_filtered, jackets_filtered])

In [13]:
if not os.path.isfile('filtered_df.csv'):
    filtered_df.to_csv('filtered_df.csv')

In [14]:
classes

['shoe', 'trousers', 'jacket', 'sweatshirt']

In [18]:
for current_class in classes:
    filtered_df[current_class] = (filtered_df["class"] == current_class).astype('int32')

In [23]:
filtered_df.head()

,fname,class,shoe,trousers,jacket,sweatshirt
0,15319.jpg,shoe,1,0,0,0
1,15324.jpg,shoe,1,0,0,0
2,15347.jpg,shoe,1,0,0,0
3,15356.jpg,shoe,1,0,0,0
4,15370.jpg,shoe,1,0,0,0


In [24]:
train_df, test_df = sklearn.model_selection.train_test_split(filtered_df, test_size=0.2, random_state=1000)

In [25]:
train_df, validation_df = sklearn.model_selection.train_test_split(train_df, test_size=0.15, random_state=1000)

In [26]:
train_df

,fname,class,shoe,trousers,jacket,sweatshirt
30,15882.jpg,shoe,1,0,0,0
94,7381.jpg,trousers,0,1,0,0
104,5241.jpg,sweatshirt,0,0,0,1
78,8202.jpg,trousers,0,1,0,0
22,1848.jpg,jacket,0,0,1,0
...,...,...,...,...,...,...
126,17402.jpg,shoe,1,0,0,0
97,7885.jpg,trousers,0,1,0,0
81,3861.jpg,jacket,0,0,1,0
66,16490.jpg,shoe,1,0,0,0


In [27]:
def loadImages(pathlist, basedir='.'):
    size = len(pathlist)

    batcharr = np.zeros(shape=(size, 300, 300, 3))

    for i in range(0,size):

        img_path = os.path.join(basedir, pathlist[i])

        im = tf.keras.preprocessing.image.load_img(
            img_path,
            target_size=(300, 300, 3)
        )
        imarr = tf.keras.preprocessing.image.img_to_array(im)
        imarr = tf.keras.applications.efficientnet.preprocess_input(imarr)

        batcharr[i] = imarr

    return batcharr

In [28]:
train_df["fname"].values[0]

'15882.jpg'

In [29]:
imgs_loaded = loadImages(train_df["fname"].values, basedir='../train')

In [35]:
imgs_loaded_validation = loadImages(validation_df["fname"].values, basedir='../train')

In [36]:
validation_y = validation_df[["shoe", "trousers", "jacket", "sweatshirt"]].values

In [30]:
train_y = train_df[["shoe", "trousers", "jacket", "sweatshirt"]].values

In [37]:
hist = mymodel.fit(imgs_loaded, train_y, epochs=5, verbose=True, batch_size=32,
                    validation_data=(imgs_loaded_validation, validation_y))

13/13 [==============================] - 151s 12s/step - loss: 0.3155 - accuracy: 0.9124 - val_loss: 0.2648 - val_accuracy: 0.9589
